# Decision Tree  
## How the Algorithm Works

**The process begins with all examples placed at the root node. Then:**

- **For each available feature, we calculate the information gain and select the one with the highest value.**  
- **We split the dataset based on the selected feature.**  
- **This process repeats recursively until a stopping condition is met.**

---

## Key Concepts

### Entropy  

**Entropy is a measure of impurity or disorder in a dataset.**  

Entropy formula:
$$
H = -\sum p_i \log_2 p_i
$$

Where:
- $p_i$ — the proportion of examples belonging to class $i$ in the node.

---

### Information Gain

**Information gain measures the reduction in entropy after a dataset is split.**  
**We aim to choose the split that results in the highest information gain.**

Formula:
$$
\text{Information Gain} = H(\text{parent}) - \left( w_\text{left} \cdot H(\text{left}) + w_\text{right} \cdot H(\text{right}) \right)
$$

Where:
- $H(\text{parent})$ — entropy of the node before the split  
- $H(\text{left})$, $H(\text{right})$ — entropies of the left and right branches  
- $w_\text{left}$, $w_\text{right}$ — proportions of examples in the left and right branches


In [1]:
import numpy as np
import pandas as pd
import time
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.datasets import make_classification

from sklearn.tree import DecisionTreeClassifier as SklearnDecisionTree
from sklearn.metrics import accuracy_score, balanced_accuracy_score

from mlfs.decision_tree import DecisionTree as CustomDecisionTree
from mlfs.metrics import (
    accuracy as custom_accuracy,
    balanced_accuracy as custom_balanced_accuracy
)

from mlfs.preprocessing import train_test_split, standardize


In [2]:
df = pd.read_csv("../data/breast-cancer.csv")
df

In [3]:
df.drop('id', axis=1, inplace=True) 

In [4]:
class_counts = df['diagnosis'].value_counts()
labels = ['Benign', 'Malignant']
sizes = [class_counts[0], class_counts[1]]
colors = ['#99ff99','#339933']

plt.figure(figsize=(6,6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', colors=colors, startangle=140)
plt.title('Diagnosis Distribution')
plt.axis('equal')
plt.show()


## From this plot we conclude that:
* **Data isn't balanced, accuracy wont be a good evaluation metric for this dataset**

In [5]:
sns.set(style="whitegrid")
features = df.columns[1:6]  
fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(20, 5))

for i, col in enumerate(features):
    sns.boxplot(x='diagnosis', y=col, data=df, ax=axes[i], palette="Set2")
    axes[i].set_title(col)
    axes[i].set_xlabel('')
    axes[i].set_ylabel('')

fig.suptitle("Boxplots for Selected Features")
plt.tight_layout()
plt.show()


In [6]:
for column in  df.drop('diagnosis',axis=1).columns[5:10]:
    fig = px.scatter(data_frame=df,color='diagnosis',x=column,color_discrete_sequence=['#007500','#5CFF5C'],)
    fig.show()

<a id="4"></a>
<h1 style='background:#00EFFF;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '

# Data Preprocessing

In [7]:
df['diagnosis'] = (df['diagnosis'] == 'M').astype(int)
corr = df.corr()
plt.figure(figsize=(20,20))
sns.heatmap(corr, cmap='viridis_r',annot=True)
plt.show()

## From this plot we conclude that:
* **Some features aren't correlated with the target maybe we should remove them**

In [8]:
notincluded_columns = abs(corr['diagnosis'])[abs(corr['diagnosis'] < 0.25)]
notincluded_columns = notincluded_columns.index.tolist()
for col in notincluded_columns:
  df.drop(col, axis = 1, inplace = True)

In [9]:
print(notincluded_columns)

In [10]:
X = df.drop('diagnosis', axis = 1).values
y = df['diagnosis']
print('Shape of X', X.shape)
print('Shape of y', y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train, mean, std = standardize(X_train, return_params=True)
X_test = (X_test - mean) / std

<a id="4"></a>
<h1 style='background:#00EFFF;border:0; color:black;
    box-shadow: 10px 10px 5px 0px rgba(0,0,0,0.75);
    transform: rotateX(10deg);
    '

# Comparing

In [12]:
def benchmark_decision_tree_custom_vs_sklearn(X, y, n_repeats=5):
    """
    Benchmarks training and prediction times for custom and sklearn decision tree implementations.

    Parameters
    ----------
    X : np.ndarray or pd.DataFrame
        Feature matrix.
    y : np.ndarray or pd.Series
        Target vector.
    n_repeats : int
        Number of times to repeat the measurement (for averaging).

    Returns
    -------
    pd.DataFrame
        DataFrame with average fit and predict times for both models.
    """

    custom_fit_times = []
    custom_predict_times = []
    sklearn_fit_times = []
    sklearn_predict_times = []

    for _ in range(n_repeats):
        custom_model = CustomDecisionTree(min_samples=2, max_depth=5)

        start = time.time()
        custom_model.fit(X, y)
        custom_fit_times.append(time.time() - start)

        start = time.time()
        custom_model.predict(X)
        custom_predict_times.append(time.time() - start)

        sklearn_model = SklearnDecisionTree(max_depth=5)

        start = time.time()
        sklearn_model.fit(X, y)
        sklearn_fit_times.append(time.time() - start)

        start = time.time()
        sklearn_model.predict(X)
        sklearn_predict_times.append(time.time() - start)

    results = pd.DataFrame({
        'Model': ['CustomDecisionTree', 'SklearnDecisionTree'],
        'FitTime': [np.mean(custom_fit_times), np.mean(sklearn_fit_times)],
        'PredictTime': [np.mean(custom_predict_times), np.mean(sklearn_predict_times)]
    })

    return results


In [13]:
df_results = benchmark_decision_tree_custom_vs_sklearn(X_train, y_train)
display(df_results)

### Benchmark Results: Custom vs Sklearn Decision Tree


#### Analysis

- **Training Time**: The custom implementation, written from scratch in Python, naturally incurs higher training time compared to scikit-learn’s optimized C-based backend. This trade-off is expected in educational or prototype implementations that prioritize clarity and algorithmic transparency over raw performance.

- **Prediction Time**: While also slightly higher in the custom version, the prediction step remains fast and efficient for typical dataset sizes.

#### Conclusion

This benchmark highlights the performance differences between a reference implementation and a production-grade library. The custom decision tree was developed to deepen understanding of core decision tree logic and data structures, and serves as a solid foundation for future optimization work. Despite the performance gap, the implementation demonstrates correctness and functional parity with sklearn’s API.


In [14]:
def benchmark_varying_sample_sizes(sample_sizes, n_features=20, n_classes=2, n_repeats=3):
    all_results = []

    for n_samples in sample_sizes:
        print(f"Benchmarking for {n_samples} samples...")
        X, y = make_classification(n_samples=n_samples,
                                   n_features=n_features,
                                   n_informative=int(n_features * 0.6),
                                   n_redundant=int(n_features * 0.2),
                                   n_classes=n_classes,
                                   random_state=42)
        results = benchmark_decision_tree_custom_vs_sklearn(X, y, n_repeats=n_repeats)
        results['sample_size'] = n_samples
        all_results.append(results)

    df = pd.concat(all_results, ignore_index=True)
    return df

def plot_benchmark_results(df):
    sns.set(style="whitegrid")
    plt.figure(figsize=(14, 6))

    plt.subplot(1, 2, 1)
    sns.lineplot(data=df, x='sample_size', y='FitTime', hue='Model', marker='o')
    plt.title('Training Time vs Number of Samples')
    plt.xlabel('Number of Samples')
    plt.ylabel('Average Training Time [s]')
    plt.xscale('log')

    plt.subplot(1, 2, 2)
    sns.lineplot(data=df, x='sample_size', y='PredictTime', hue='Model', marker='o')
    plt.title('Prediction Time vs Number of Samples')
    plt.xlabel('Number of Samples')
    plt.ylabel('Average Prediction Time [s]')
    plt.xscale('log')

    plt.tight_layout()
    plt.show()

sample_sizes = [100, 500, 1000, 5000, 10000]
df_results = benchmark_varying_sample_sizes(sample_sizes, n_features=20, n_classes=2, n_repeats=3)
plot_benchmark_results(df_results)

In [15]:
custom_model = CustomDecisionTree(min_samples=2, max_depth=5)
custom_model.fit(X_train, y_train)

sk_model = SklearnDecisionTree(max_depth=5)
sk_model.fit(X_train, y_train)

y_pred_custom = custom_model.predict(X_test)
y_pred_sk     = sk_model.predict(X_test)

y_true = np.array(y_test).ravel()

acc_c  = custom_accuracy(y_true, y_pred_custom)
balacc_c = custom_balanced_accuracy(y_true, y_pred_custom)

acc_s  = accuracy_score(y_true, y_pred_sk)
balacc_s = balanced_accuracy_score(y_true, y_pred_sk)

df_results = pd.DataFrame({
    'Model':            ['Custom',       'Sklearn'],
    'Accuracy':         [acc_c,          acc_s],
    'Balanced Accuracy':[balacc_c,       balacc_s]
})

display(df_results)
